In [ ]:
import consul
import sys
import uuid

In [ ]:
def register_service(service_name, service_ip, service_port, consul_ip, consul_port):
    consul_client = consul.Consul(host=consul_ip, port=consul_port)

    service_id = f"{service_name}-{str(uuid.uuid4())[:4]}"

    consul_client.agent.service.register(
    name=service_name,
    service_id=service_id,
    address=service_ip,
    port=service_port,
    check=consul.Check.http(
        url=f"http://{service_ip}:{consul_ip}/health",
        interval="10s",
        timeout="1s",
        deregister="10m"
    )
)

In [ ]:
SERVICE_NAME = sys.argv[0][:-3]
SERVICE_ID = f"{SERVICE_NAME}-{str(uuid.uuid4())[:4]}"

CONSUL_IP = "localhost"
CONSUL_PORT = 8500

SERVICE_IP = "host.docker.internal"
SERVICE_PORT = 5000

In [ ]:
consul_client = consul.Consul(host=CONSUL_IP, port=CONSUL_PORT)
try:
    leader = c.status.leader()
    print(f"Consul is available. Leader: {leader}")
except Exception as e:
    print(f"Failed to connect to Consul: {e}")

In [ ]:
register_service(SERVICE_NAME, SERVICE_IP, SERVICE_PORT, CONSUL_IP, CONSUL_PORT)

### Get all services

In [ ]:
services = c.agent.services()
for service_id, service_info in services.items():
    print(f"Service ID: {service_id}")
    print(f"Service Name: {service_info['Service']}")
    print(f"Service Address (IP): {service_info['Address']}")
    print(f"Service Port: {service_info['Port']}")
    print("-" * 30)

In [ ]:
services = c.agent.services()
def get_service_ips_consul(service_name: str):
    service_name = service_name.replace("-", "_")
    result_ips = []

    for service_id, service_info in services.items():
        if service_name == service_info["Service"][:-3]:
            result_ips.append(f"{service_info['Address']}:{service_info['Port']}")

    return result_ips

In [ ]:
print(get_service_ips_consul("messages-service"))


### Set a key

In [ ]:
# Set a key
key = 'a'
value = 'b'

c.kv.put(key, value)

print(f"Key '{key}' set successfully!")

### Get a key

In [ ]:
index, data = c.kv.get('a')

if data is not None:
    value = data['Value'].decode('utf-8')  # Must decode from bytes
    print(f"Key value: {value}")
else:
    print("Key not found.")


---

In [6]:
import hazelcast

hazelcast_url = "0.0.0.0:5703"

client = hazelcast.HazelcastClient(cluster_members=[hazelcast_url])
distributed_map = client.get_map("my-distributed-map").blocking()
distributed_map.set(1 ,2)

In [ ]:
messages = distributed_map.entry_set()
print(messages)

[(1, 2)]


In [16]:
import requests
st = 'st'

data = {"uuid": "76344bff-b269-46dc-91ad-ee9577f15bd1", "msg": "I am message 6"}
logging_service_url = "http://127.0.0.1:5001"
response = requests.post(logging_service_url, json=data, timeout=3)
print(response.status_code)


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5001): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7eec8d7d69b0>: Failed to establish a new connection: [Errno 111] Connection refused'))